In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def get_class_paths(path):
  classes = []
  class_paths = []

  # Iterate through directories in the training path
  for label in os.listdir(path):
    label_path = os.path.join(path, label)

    # Check if it's a directory
    if os.path.isdir(label_path):
      # Iterate through images in the label directory
      for image in os.listdir(label_path):
        image_path = os.path.join(label_path, image)

        # Add class and path to restrictive lists
        classes.append(label)
        class_paths.append(image_path)

  # Create a DataFrame with the collected data
  df = pd.DataFrame({
      'Class Path': class_paths,
      'Class': classes
  })

  return df

In [3]:
tr_df = get_class_paths("Training")

In [ ]:
tr_df

In [5]:
ts_df = get_class_paths("Testing")

In [ ]:
ts_df


In [ ]:
plt.figure(figsize=(15, 7))
ax = sns.countplot(data=tr_df, x=tr_df["Class"])

In [ ]:
plt.figure(figsize=(15, 7))
ax = sns.countplot(data=ts_df, x=ts_df["Class"])

In [12]:
# !pip install scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
valid_df, ts_df = train_test_split(ts_df, train_size=0.5, stratify=ts_df['Class'])

In [ ]:
valid_df

In [ ]:
ts_df

In [13]:
batch_size = 32

img_size = (299, 299)

image_generator = ImageDataGenerator(rescale=1/255, brightness_range=(0.8, 1.2))

ts_gen = ImageDataGenerator(rescale=1/255)

In [ ]:
tr_gen = image_generator.flow_from_dataframe(tr_df, x_col='Class Path', y_col='Class', batch_size=batch_size, target_size=img_size)

valid_gen = image_generator.flow_from_dataframe(valid_df, x_col='Class Path', y_col='Class', batch_size=batch_size, target_size=img_size)

ts_gen = ts_gen.flow_from_dataframe(ts_df, x_col='Class Path', y_col='Class', batch_size=16, target_size=img_size, shuffle=False)

In [ ]:
plt.figure(figsize=(20, 20))
for i in range (16):
  plt.subplot(4, 4, i+1)
  batch = next(tr_gen)
  image = batch[0][0]
  label = batch[1][0]
  plt.imshow(image)

  # Get the class index
  class_index = np.argmax(label)

  # Get the list of class names and class indeies
  class_names = list(tr_gen.class_indices.keys())
  class_indices = list(tr_gen.class_indices.values())

  # Find the index of the class_index in the list of indices
  index_position = class_indices.index(class_index)

  # Get the class name using the index position
  class_name = class_names[index_position]

  plt.title(f"Class: {class_name}")
  plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
img_shape = (299, 299, 3)

base_model = tf.keras.applications.Xception(include_top=False, weights="imagenet", input_shape= img_shape, pooling='max')

model = Sequential([base_model, Flatten(), Dropout(rate= 0.3), Dense(128, activation= 'relu'), Dropout(rate= 0.25), Dense(4, activation= 'softmax')])

In [17]:
model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy', Precision(), Recall()])

In [ ]:
hist = model.fit(tr_gen, epochs=5, validation_data=valid_gen)

In [ ]:
hist.history.keys()

In [ ]:
  # Get training and validation metrics from history
metrics = ['accuracy', 'precision', 'loss', 'recall']  
tr_metrics = {m: hist.history[m] for m in metrics}
val_metrics = {m: hist.history[f'val_{m}'] for m in metrics}

# Find best epochs and values
best_epochs = {}
best_values = {}
for m in metrics:
    if m == 'loss':
        idx = np.argmin(val_metrics[m])
    else:
        idx = np.argmax(val_metrics[m])
    best_epochs[m] = idx + 1
    best_values[m] = val_metrics[m][idx]

# Plot metrics
plt.figure(figsize=(20, 12))
plt.style.use('fivethirtyeight')

for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 2, i)
    epochs = range(1, len(tr_metrics[metric]) + 1)
    
    plt.plot(epochs, tr_metrics[metric], 'r', label=f'Training {metric}')
    plt.plot(epochs, val_metrics[metric], 'g', label=f'Validation {metric}')
    plt.scatter(best_epochs[metric], best_values[metric], s=150, c='blue', 
                label=f'Best epoch = {best_epochs[metric]}')
    
    plt.title(f'Training and Validation {metric.title()}')
    plt.xlabel('Epochs')
    plt.ylabel(metric.title())
    plt.legend()
    plt.grid(True)

plt.suptitle('Model Training Metrics Over Epochs', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
train_score = model.evaluate(tr_gen, verbose=1)
valid_score = model.evaluate(valid_gen, verbose=1)
test_score = model.evaluate(ts_gen, verbose=1)

print(f"Training Accuracy: {train_score[1]*100:.2f}%")
print(f"Validation Loss: {valid_score[0]:.4f}")
print(f"\n\nValidation Accuracy: {valid_score[1]*100:.2f}%")
print(f"Validation Loss: {valid_score[0]:.4f}")
print(f"\n\nTest Accuracy: {test_score[1]*100:.2f}%")
print(f"Testing Loss: {test_score[0]:.4f}")


In [ ]:
preds = model.predict(ts_gen)
y_pred = np.argmax(preds, axis=1)

class_dict = {
  0: 'glioma',
  1: 'meningioma',
  2: 'no_tumor',
  3: 'pituitary'
}

# Then create a display the confusion matrix
cm = confusion_matrix(ts_gen.classes, y_pred)
labels = list(class_dict.keys())
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [20]:
from PIL import Image

def predict(img_path: str) -> None:
  # Get class labels
  labels = list(class_dict.keys())

  # Load a process image
  img = Image.open(img_path)
  resized_image = img.resize((299, 299))
  img_array = np.asarray(resized_image)
  img_array = np.expand_dims(img_array, axis=0) / 255.0

  # Get model predictions
  predictions = model.predict(img_array)
  probabilities = list(predictions[0])

  # Get Predicted class
  predicted_class_idx = np.argmax(probabilities)
  predicted_class = class_dict[predicted_class_idx]

  # Plot original image
  plt.subplot(2, 1, 1)
  plt.imshow(resized_image)
  plt.title(f"Input MRI Image\nPredicted: {predicted_class}")

  # Plot prediction probabilities
  plt.subplot(2, 1, 2)
  bars = plt.barh(labels, probabilities)
  plt.xlabel("Probability", fontsize=15)
  plt.title("Class Probabilities")

  # Add probability labels to bars
  ax = plt.gca()
  ax.bar_label(bars, fmt='%.2f')

  plt.tight_layout()
  plt.show()

  print(f"\nPredicted tumor type: {predicted_class}")


In [ ]:
predict("Testing/meningioma/Te-me_0011.jpg")

In [ ]:
predict("Testing/meningioma/Te-me_0015.jpg")

In [ ]:
predict("Testing/glioma/Te-gl_0010.jpg")

In [30]:
model.save_weights("xception_model.weights.h5")


In [13]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers


In [ ]:
batch_size = 16

img_size = (224, 224)


image_generator = ImageDataGenerator(rescale=1/255, brightness_range=(0.8, 1.2))

ts_gen = ImageDataGenerator(rescale=1/255)

tr_gen = image_generator.flow_from_dataframe(tr_df, x_col='Class Path', y_col='Class', batch_size=batch_size, target_size=img_size)

valid_gen = image_generator.flow_from_dataframe(valid_df, x_col='Class Path', y_col='Class', batch_size=batch_size, target_size=img_size)

ts_gen = ts_gen.flow_from_dataframe(ts_df, x_col='Class Path', y_col='Class', batch_size=16, target_size=img_size, shuffle=False)

In [ ]:
# Create a sequential model

cnm_model = Sequential()

# Convolution Layers
cnm_model.add(Conv2D(512, (3, 3), padding= 'same', input_shape=(224, 224, 3), activation= 'relu'))
cnm_model.add(MaxPooling2D(pool_size=(2, 2)))

cnm_model.add(Conv2D(256, (3, 3), padding= 'same', activation= 'relu'))
cnm_model.add(MaxPooling2D(pool_size=(2, 2)))
cnm_model.add(Dropout(0.25))

cnm_model.add(Conv2D(128, (3, 3), padding= 'same', activation= 'relu'))
cnm_model.add(MaxPooling2D(pool_size=(2, 2)))
cnm_model.add(Dropout(0.25))

cnm_model.add(Conv2D(64, (3, 3), padding= 'same', activation= 'relu'))
cnm_model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output for fully connected layers
cnm_model.add(Flatten())

# Fully connected layers
cnm_model.add(Dense(256, activation= 'relu', kernel_regularizer= regularizers.l2(0.01)))
cnm_model.add(Dropout(0.35))

# Output layer with 4 neurons for the 4 classes
cnm_model.add(Dense(4, activation='softmax'))

# Compile the model
cnm_model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy', Precision(), Recall()])

# Display the model summary
cnm_model.summary()

In [ ]:
history = cnm_model.fit(tr_gen, epochs=5, validation_data=valid_gen)

In [ ]:
history.history.keys()

In [ ]:
  # Get training and validation metrics from history
metrics = ['accuracy', 'precision_2', 'loss', 'recall_2']  
tr_metrics = {m: history.history[m] for m in metrics}
val_metrics = {m: history.history[f'val_{m}'] for m in metrics}

# Find best epochs and values
best_epochs = {}
best_values = {}
for m in metrics:
    if m == 'loss':
        idx = np.argmin(val_metrics[m])
    else:
        idx = np.argmax(val_metrics[m])
    best_epochs[m] = idx + 1
    best_values[m] = val_metrics[m][idx]

# Plot metrics
plt.figure(figsize=(20, 12))
plt.style.use('fivethirtyeight')

for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 2, i)
    epochs = range(1, len(tr_metrics[metric]) + 1)
    
    plt.plot(epochs, tr_metrics[metric], 'r', label=f'Training {metric}')
    plt.plot(epochs, val_metrics[metric], 'g', label=f'Validation {metric}')
    plt.scatter(best_epochs[metric], best_values[metric], s=150, c='blue', 
                label=f'Best epoch = {best_epochs[metric]}')
    
    plt.title(f'Training and Validation {metric.title()}')
    plt.xlabel('Epochs')
    plt.ylabel(metric.title())
    plt.legend()
    plt.grid(True)

plt.suptitle('Model Training Metrics Over Epochs', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
train_score = cnm_model.evaluate(tr_gen, verbose=1)
valid_score = cnm_model.evaluate(valid_gen, verbose=1)
test_score = cnm_model.evaluate(ts_gen, verbose=1)

print(f"Training Accuracy: {train_score[1]*100:.2f}%")
print(f"Validation Loss: {valid_score[0]:.4f}")
print(f"\n\nValidation Accuracy: {valid_score[1]*100:.2f}%")
print(f"Validation Loss: {valid_score[0]:.4f}")
print(f"\n\nTest Accuracy: {test_score[1]*100:.2f}%")
print(f"Testing Loss: {test_score[0]:.4f}")


In [ ]:
preds = cnm_model.predict(ts_gen)
y_pred = np.argmax(preds, axis=1)

class_dict = {
  0: 'glioma',
  1: 'meningioma',
  2: 'no_tumor',
  3: 'pituitary'
}

# Then create a display the confusion matrix
cm = confusion_matrix(ts_gen.classes, y_pred)
labels = list(class_dict.keys())
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
clr = classification_report(ts_gen.classes, y_pred)
print(clr)

In [71]:
cnm_model.save_weights("cnm_model.weights.h5")

# Streamlit app

In [2]:
!pip install streamlit pyngrok python-dotenv



In [5]:
import os
from threading import Thread
from pyngrok import ngrok


In [6]:
from dotenv import load_dotenv
load_dotenv()

ngrok_token = os.getenv('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(ngrok_token)


In [7]:
def run_streamlit():
  os.system("streamlit run main.py --server.port 8501")

In [8]:
%%writefile main.py

import openai
from openai import OpenAI
import base64
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import plotly.graph_objects as go
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
import PIL.Image
import os
from dotenv import load_dotenv
load_dotenv()

output_dir = "saliency_maps"
os.makedirs(output_dir, exist_ok=True)

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def load_model(model_path):
    try:
        # Load the entire model
        model = tf.keras.models.load_model(model_path)
        return model
    except:
        # If loading fails, recreate the model architecture and load weights
        model = Sequential([
            Conv2D(512, (3, 3), padding='same', input_shape=(224, 224, 3), activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(256, (3, 3), padding='same', activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            Conv2D(128, (3, 3), padding='same', activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            Conv2D(64, (3, 3), padding='same', activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
            Dropout(0.35),
            Dense(4, activation='softmax')
        ])
        
        # Compile the model
        model.compile(
            Adamax(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy', Precision(), Recall()]
        )
        
        # Load weights
        model.load_weights(model_path)
        return model

def generate_explanation(model, model_prediction, confidence):

  prompt = f""" 
  You are an expert neurologist analyzing a brain tumor MRI scan with an overlaid saliency map. This map highlights in light cyan the areas the deep learning model focused on to classify the brain tumor. The model has classified the tumor as '{model_prediction}' with a confidence level of {confidence*100}%.

  In your analysis:
  - Identify the specific brain regions in the MRI that the model is concentrating on, based on the light cyan areas in the saliency map, and describe any significant anatomical landmarks or structures that fall within these highlighted regions.
  - Discuss plausible reasons why the model focused on these specific regions for the given classification, considering the typical tumor characteristics and growth patterns associated with glioma, meningioma, pituitary tumors, or no tumor.
  - Offer an insight into how the highlighted areas might correlate with the model's predicted tumor type.

  Do not mention the model at all in your explanation.
  Do not mention the work 'model' in your explanation.

  Keep your explanation concise and limited to 4 sentences. Ensure clarity and precision in your analysis by verifying each step logically.
    """

  # img = PIL.Image.open(img_path)
  image_path = os.path.join(output_dir, uploaded_file.name)
  with open(image_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

  response = client.chat.completions.create(
        model="gpt-4o",
        # model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{encoded_image}"
                        }
                    }
                ]
            }
        ],
    )
    
  return response.choices[0].message.content

def generate_saliency_map(model, img_array, class_index, img_size):
  with tf.GradientTape() as tape:
    img_tensor = tf.convert_to_tensor(img_array)
    tape.watch(img_tensor)
    predictions = model(img_tensor)
    target_class = predictions[:, class_index]

    gradients = tape.gradient(target_class, img_tensor)
    gradients = tf.math.abs(gradients)
    gradients = tf.reduce_max(gradients, axis=-1)
    gradients = gradients.numpy().squeeze()

    # Resize gradients to match the original image size
    gradients = cv2.resize(gradients, img_size)

    # Create a circular mask for the brain area
    center = (gradients.shape[0] // 2, gradients.shape[1] // 2)
    radius = min(center[0], center[1]) - 10
    y, x = np.ogrid[:gradients.shape[0], :gradients.shape[1]]
    mask = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= radius ** 2

    # Apply the mask to gradients
    gradients = gradients * mask

    # Normalize only the brain area
    brain_gradients = gradients[mask]
    if brain_gradients.max() > brain_gradients.min():
      brain_gradients = (brain_gradients - brain_gradients.min()) / (brain_gradients.max() - brain_gradients.min())
    gradients[mask] = brain_gradients

    # Apply a higher threshold
    threshold = np.percentile(gradients[mask], 80)
    gradients[gradients < threshold] = 0

    # Apply more aggressive smoothing
    gradients = cv2.GaussianBlur(gradients, (11, 11), 0)

    # Create a heatmap overlay with enhanced contrast
    heatmap = cv2.applyColorMap(np.uint8(255 * gradients), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    # Resize heatmap to match the original image size
    heatmap = cv2.resize(heatmap, img_size)

    # Superimpose the heatmap on original image with increased opacity
    original_image = image.img_to_array(img)
    superimposed_img = heatmap * 0.7 + original_image * 0.3
    superimposed_img = superimposed_img.astype(np.uint8)

    img_path = os.path.join(output_dir, uploaded_file.name)
    with open(img_path, 'wb') as f:
      f.write(uploaded_file.getbuffer())

    saliency_map_path = f"saliency_maps/{uploaded_file.name}"

    # Save the saliency map
    cv2.imwrite(saliency_map_path, cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))

    return superimposed_img

def load_xception_model(model_path):
  img_shape = (299, 299, 3)
  base_model = tf.keras.applications.Xception(include_top=False, weights="imagenet", input_shape= img_shape, pooling='max')

  model = Sequential([base_model, Flatten(), Dropout(rate= 0.3), Dense(128, activation= 'relu'), Dropout(rate= 0.25), Dense(4, activation='softmax')])

  model.build((None,) + img_shape)

  # Compile the model
  model.compile(Adamax(learning_rate= 0.001), loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

  model.load_weights(model_path)

  return model

st.title("Brain Tumor Classification")

# Upload Image
st.write("Upload an image of a brain MRI scan to classify the type of tumor.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:

  selected_model = st.radio(
    "Select Model",
    ("Transfer Learning - Xception", "Custom CNN")  
  )

  if selected_model == "Transfer Learning - Xception":
    model = load_xception_model("xception_model.weights.h5")
    img_size = (299, 299)
  else:
    model = load_model("cnm_model.weights.h5")
    img_size = (224, 224)

  labels = ['Glioma', 'Meningioma', 'No_tumor', 'Pituitary']
  img = image.load_img(uploaded_file, target_size=img_size)
  img_array = image.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = img_array / 255.0

  predictions = model.predict(img_array)

  # Get the class with the highest probability
  class_index = np.argmax(predictions[0])
  result = labels[class_index]

  st.write(f"Predicted Class: {result}")
  st.write("Predictions:")
  for label, prob in zip(labels, predictions[0]):
    st.write(f"{label}: {prob:.4f}")

  # Generate saliency map
  saliency_map = generate_saliency_map(model, img_array, class_index, img_size)

  col1, col2 = st.columns(2)
  with col1:
    st.image(uploaded_file, caption="Uploaded Image", use_container_width=True)
  with col2:
    st.image(saliency_map, caption="Saliency Map", use_container_width=True)

  st.write("## Classification Results:")

  result_container = st.container()
  result_container = st.container()
  result_container.markdown(
    f""" 
    <div style="background-color: #000000; color: #ffffff; padding: 30px; border-radius: 15px">
      <div style="display: flex; justify-content: space-between; align-items: center;">
        <div style="flex: 1; text-align: center;">
          <h3 style="color: #ffffff; margin-bottom: 10px; font-size: 20px;">Prediction</h3>
          <p style="font-size: 36px; font-weight: 800; color: #FF0000; margin: 0;">
            {result}
          </p>
        </div>
        <div style="width: 2px; height: 80px; background-color: #ffffff; margin: 0 20px;"></div>
        <div style="flex: 1; text-align: center;">
          <h3 style="color: #ffffff; margin-bottom: 10px; font-size: 20px;">Confidence</h3>
          <p style="font-size: 36px; font-weight: 800; color: #2196F3; margin: 0">
            {predictions[0][class_index]:.4%}
          </p>
        </div>
        
      </div>
    </div>
    """,
    unsafe_allow_html= True
  )

  # Prepare data for Plotly chart
  probabilities = predictions[0]
  sorted_indices = np.argsort(probabilities)[::-1]
  sorted_labels = [labels[i] for i in sorted_indices]
  sorted_probabilities = probabilities[sorted_indices]

  # Create a Plotly chart
  fig = go.Figure(go.Bar(
  x=sorted_probabilities,
  y=sorted_labels,
  orientation= 'h',
  marker_color=['red' if label == result else 'blue' for label in sorted_labels]
  ))

  # Customize the chart layout
  fig.update_layout(
  title='Probabilities for each class',
  xaxis_title='Probability',
  yaxis_title='Class',
  height= 400,
  width=600,
  yaxis=dict(autorange= 'reversed')
  )

  # Add value labels to the bars
  for i, prob in enumerate(sorted_probabilities):
    fig.add_annotation(
      x=prob,
      y=i,
      text=f'{prob:.4f}',
      showarrow= False,
      xanchor='left',
      xshift=5
    )

  # Display the Plotly chart
  st.plotly_chart(fig)

  saliency_map_path = f'saliency_maps/{uploaded_file.name}'

  explanation = generate_explanation(saliency_map_path, result, predictions[0][class_index])

  st.write("## Explanation:")
  st.write(explanation)



Overwriting main.py


In [9]:
thread = Thread(target=run_streamlit)
thread.start()


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.189.0.83:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            


2024-11-11 17:40:11.331705: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-11-11 17:40:11.331744: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-11 17:40:11.331748: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-11 17:40:11.332064: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-11 17:40:11.332077: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/paulc1/miniforge3/envs/tumor-classification/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamax', bec

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [1]:
public_url = ngrok.connect(addr="8501", proto="http", bind_tls=True)

tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
  print(f"Closing tunnel: {tunnel.public_url} -> {tunnel.config['addr']}")
  ngrok.disconnect(tunnel.public_url)
print("\n")

public_url = ngrok.connect(addr="8501", proto="http", bind_tls=True)

print("Public URL:", public_url)

NameError: name 'ngrok' is not defined